In [1]:
import pandas as pd
import json
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\o0o0o0o\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\o0o0o0o\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# Leemos el json y lo guardamos en un string
with open('src/australian_user_reviews.json', encoding='UTF-8') as f:
    json_string = f.read()

In [3]:
#Reemplazamos las comillas dobles por simples y luego las simples por dobles para que el json sea válido
json_string = json_string.replace("\"", "'")
json_string = json_string.replace("\\'", "'")
json_string = json_string.replace("\\", "'")
json_string = json_string.replace("{'user_id': '", '{"user_id": "')
json_string = json_string.replace("', 'user_url': '", '", "user_url": "')
json_string = json_string.replace("', 'reviews': [{'funny': '", '", "reviews": [{"funny": "')
json_string = json_string.replace("', 'reviews': []}", '", "reviews": []}')
json_string = json_string.replace("'}, {'funny': '", '"}, {"funny": "')
json_string = json_string.replace("', 'posted': '", '", "posted": "')
json_string = json_string.replace("', 'last_edited': '", '", "last_edited": "')
json_string = json_string.replace("', 'item_id': '", '", "item_id": "')
json_string = json_string.replace("', 'helpful': '", '", "helpful": "')
json_string = json_string.replace("', 'recommend': True, 'review': '", ' ", "recommend": "True", "review": "')
json_string = json_string.replace("', 'recommend': False, 'review': '", ' ", "recommend": "False", "review": "')
json_string = json_string.replace("'}]}", '"}]}')  
json_string = json_string.replace("\\xa0", ' ')


In [4]:
# Recorremos el json y lo convertimos en un array. La columna reviews se desanida y se crea un nuevo array,
# agregando el user_id a cada review
data_array = []
reviews_array = []
for line in json_string.splitlines():
    line = json.loads(line)
    
    if line['reviews'] != []:
        for review in line['reviews']:
                review["user_id"] = line['user_id']
                reviews_array.append(review)
    data_array.append(line)

In [5]:
# Creamos los dataframes
reviews_df = pd.DataFrame(reviews_array)
users_df = pd.DataFrame(data_array)

In [6]:
#Limpiamos el df de reviews
reviews_df = reviews_df.drop(columns=['funny', 'posted', 'last_edited', 'helpful'])

#Limpiamos el df de users
users_df = users_df.drop(columns=['user_url', 'reviews'])

In [7]:
#Acondicionamos el el texto de la columna review
corpus = []
for i in range(0, reviews_df.shape[0]):
  review = re.sub('[^a-zA-Z]', ' ', reviews_df['review'][i])# remplaza cualquier cosa que no sea Letras por espacios
  review = review.lower()# pasar a minusculas
  review = review.split() # se divide en plabras sin espacions
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')# eliminar palabras que no dan sentimiento
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)


In [8]:
#agregamos la columna tokenizada al df
reviews_df['review_tokenized'] = corpus

In [9]:
#Instanciamos el SentimentIntensityAnalyzer y lo aplicamos a la columna review_tokenized en una nueva columna
sia = SentimentIntensityAnalyzer()

reviews_df['sentiment_analysis'] = reviews_df['review_tokenized'].apply(lambda x: round(sia.polarity_scores(x)['compound']) + 1)

In [10]:
#Eliminamos la columna review_tokenized y review y cambiamos el nombre de la columna sentiment por review
reviews_df = reviews_df.drop(columns=['review_tokenized'])
reviews_df = reviews_df.drop(columns=['review'])

In [11]:
#Guardamos los dataframes en archivos parquet
reviews_df.to_parquet('src/cleaned/reviews.parquet', index=False)
users_df.to_parquet('src/cleaned/users.parquet', index=False)
